In [1]:
from pathlib import Path
import scanpy as sc
import scipy
from scipy.sparse import csr_matrix as csr
import os
import pandas as pd
import numpy as np

In [2]:
def preprocess_fast(sdata1, mode = 'customized',target_sum=1e4,base = 2,zero_center = True,regressout = False):
    if type(sdata1.layers['raw']) != scipy.sparse._csr.csr_matrix:
        sdata1.layers['raw'] = csr(sdata1.layers['raw'].copy())
    sdata1.X = sdata1.layers['raw'].copy()
    if mode == 'default':
        sc.pp.normalize_total(sdata1)  # normalize counts per cell
        sdata1.layers['norm'] = csr(sdata1.X.copy())
        sc.pp.log1p(sdata1)
        sdata1.layers['log1p_norm'] = csr(sdata1.X.copy())
        sc.pp.scale(sdata1,zero_center = zero_center)
        if scipy.sparse.issparse(sdata1.X): #### automatically change to non csr matrix (zero_center == True, the .X would be sparce)
            sdata1.X = sdata1.X.toarray().copy()
        sdata1.layers['log1p_norm_scaled'] = sdata1.X.copy()
        if regressout:
            sdata1.obs['total_counts'] = sdata1.layers['raw'].toarray().sum(axis=1)
            sc.pp.regress_out(sdata1, ['total_counts'])
            sdata1.layers['log1p_norm_scaled'] = sdata1.X.copy()
        return sdata1 #### sdata1.X is sdata1.layers['log1p_norm_scaled']
    elif mode == 'customized':
        if target_sum == 1e4:
            target_sum_str = '1e4'
        else:
            target_sum_str = str(target_sum)
        sc.pp.normalize_total(sdata1,target_sum=target_sum)
        sdata1.layers[f'norm{target_sum_str}'] = csr(sdata1.X.copy())
        sc.pp.log1p(sdata1,base = base)
        sdata1.layers[f'log{str(base)}_norm{target_sum_str}'] = csr(sdata1.X.copy())
        sc.pp.scale(sdata1,zero_center = zero_center)
        if scipy.sparse.issparse(sdata1.X): #### automatically change to non csr matrix (zero_center == True, the .X would be sparce)
            sdata1.X = sdata1.X.toarray().copy()
        sdata1.layers[f'log{str(base)}_norm{target_sum_str}_scaled'] = sdata1.X.copy()
        if regressout:
            sdata1.obs['total_counts'] = sdata1.layers['raw'].toarray().sum(axis=1)
            sc.pp.regress_out(sdata1, ['total_counts'])
            sdata1.layers[f'log{str(base)}_norm{target_sum_str}_scaled'] = sdata1.X.copy()
        return sdata1 #### sdata1.X is sdata1.layers[f'log{str(base)}_norm{target_sum_str}_scaled']
    else:
        print('Please set the `mode` as one of the {"default", "customized"}.')

# Processing

Here we read from a h5ad data and do the data norm.
A new adata is returned. 

In [3]:
basepath = Path("/home/unix/wangyanz/codon_usage/scRNA_aging")
data_pool = basepath.joinpath("data")

In [18]:
sdata = sc.read_h5ad('/stanley/WangLab/Data/Raw/01.SAM/01.Tabula_Muris_v1_v2/01.h5ad/Tabula_Muris_10X_v2.h5ad')

# sdata.layers['raw'] = sdata.X.copy()
# sdata = preprocess_fast(sdata,mode = 'default')
# ### sdata.layers
# ### ctype info
# sdata.write_h5ad(data_pool.joinpath("processed_Tabula_Muris_10X_v2.h5ad"))

In [19]:
sdata.obs['mouse.id'].value_counts()

3-F-56     17806
3-F-57      9821
3-M-5/6     7410
3-M-8       7299
3-M-7/8     7167
3-M-9       5178
3-M-8/9      975
Name: mouse.id, dtype: int64

### Then we group data by cell type

In [5]:
# store the gene name
gene_name = (sdata.var["gene_name"])
gene_name_file = data_pool.joinpath("gene_name_scRNA_Muris_10X_v2.csv")
if not os.path.exists(gene_name_file):
    gene_name.to_csv(gene_name_file)

In [6]:
sdata

AnnData object with n_obs × n_vars = 55656 × 23433
    obs: 'sample', 'batch', 'cell_type', 'cell_ontology_id', 'mouse.id', 'mouse.sex', 'subtissue', 'tissue', 'tissue_tSNE_1', 'tissue_tSNE_2'
    var: 'gene_name', 'mean', 'std'
    uns: 'log1p'
    layers: 'raw', 'norm', 'log1p_norm', 'log1p_norm_scaled'

In [23]:
# group the data by cell type

# sdata.obs.groupby('cell_type', group_keys=True) retuen a index
# x.index will give you the cell index
# sdata[x.index] will group the sdata by that index, each item is a small AnnData.
# by .layers["norm"], you can acess the real data.
grouped_adata = sdata.obs.groupby('cell_type').apply(lambda x: sdata[x.index].layers["norm"].mean(axis=0))
value_list = []
for i in  grouped_adata.values:
    value_list.append(i.flatten())
value_list = np.array(value_list).reshape((len(grouped_adata.index), sdata.n_vars))
    
df = pd.DataFrame(value_list, index=grouped_adata.index, columns=gene_name.index)
df.to_csv(data_pool.joinpath("cell_type_norm_mean_scRNA_Muris_10X_v2.csv"))

### Store the cell type

In [24]:
cell_type = grouped_adata.index.tolist()
with open(data_pool.joinpath("cell_types_scRNA_Muris_10X_v2.dat"), "w") as scr:
    scr.write("\n".join(cell_type))